1. Importing needed libraries

In [66]:
# importing libraries
import numpy as np
import pandas as pd

--------------------------------
2. Reading csv files, reviewing their features

In [67]:
yt = pd.read_csv('US_youtube_trending_data.csv')
cg = pd.read_csv('category_ids.csv')

In [68]:
# head of youtube data:
yt.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...


So there are 16 columns. including:
- video_id
- title
- publishedAt
- channelId
- channelTitle
- categoryId
- trending_date
- tags
- view_count
- likes
- dislikes
- comment_count
- thumbnail_link
- comments_disabled
- ratings_disabled
- description

In [69]:
cg.head()

,categoryId,category
0,2,Autos & Vehicles
1,1,Film & Animation
2,10,Music
3,15,Pets & Animals
4,17,Sports


So there are two columns:
- categoryId
- category

-------------------------------------

3. Drop unneeded features of youtube's dataset (inplace):
- video_id
- channelId
- thumbnail_link
- comments_disabled
- ratings_disabled

In [70]:
yt.drop(columns=['video_id','channelId', 'thumbnail_link', 'comments_disabled', 'ratings_disabled'], inplace = True)

In [71]:
# See the result of the dropping:
yt[100:102]

,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,description
100,WE DID IT,2020-08-09T19:54:58Z,Deji,24,2020-08-12T00:00:00Z,[None],1547751,232695,4065,12748,FOLLOW MY REDDIT: https://www.reddit.com/r/Dej...
101,"Game Theory: Minecraft, Stop Using Diamonds!",2020-08-08T18:05:14Z,The Game Theorists,20,2020-08-12T00:00:00Z,minecraft|minecraft theory|minecraft diamonds|...,2657891,154064,6509,21993,Thanks Honey for sponsoring! Get the best dea...


--------------------------------
4. In the description of this part, the author says we usually need to convert a string-type feature which is a date to a datetime-type field. 

So, initially I show how to do that:

In [72]:
pd.to_datetime(yt['publishedAt'][100:102])

100   2020-08-09 19:54:58+00:00
101   2020-08-08 18:05:14+00:00
Name: publishedAt, dtype: datetime64[ns, UTC]

See the dtype of the output, the string-type rows are shown as datetime64 :)

If we didn't have the ability to use to_datetime, equivalently, we could use df.str.extract and using regex:

In [73]:
d = yt['publishedAt'][100:105].str.extract('(\d\d\d\d-\d\d-\d\d)T(\d\d:\d\d:\d\d)Z')
d.rename(columns={0:'date', 1:'time'}, inplace=True)
d

,date,time
100,2020-08-09,19:54:58
101,2020-08-08,18:05:14
102,2020-08-10,00:15:00
103,2020-08-10,02:00:06
104,2020-08-09,21:15:00


Of course, that's just for showing how to convert, now to convert on the real data:

In [74]:
yt['date_published'] = pd.to_datetime(yt['publishedAt']).dt.date
yt['date_published'] = pd.to_datetime(yt['date_published'])
yt.head()

,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,description,date_published
0,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...,2020-08-11
1,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,"While running her own modding shop, Ramya Pare...",2020-08-11
2,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,I left youtube for a month and this is what ha...,2020-08-11
3,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,Subscribe to XXL → http://bit.ly/subscribe-xxl...,2020-08-11
4,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,Transforming The LaBrant Family's empty white ...,2020-08-11


In [75]:
yt['time_published'] = pd.to_datetime(yt['publishedAt']).dt.time
yt[100:102]

,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,description,date_published,time_published
100,WE DID IT,2020-08-09T19:54:58Z,Deji,24,2020-08-12T00:00:00Z,[None],1547751,232695,4065,12748,FOLLOW MY REDDIT: https://www.reddit.com/r/Dej...,2020-08-09,19:54:58
101,"Game Theory: Minecraft, Stop Using Diamonds!",2020-08-08T18:05:14Z,The Game Theorists,20,2020-08-12T00:00:00Z,minecraft|minecraft theory|minecraft diamonds|...,2657891,154064,6509,21993,Thanks Honey for sponsoring! Get the best dea...,2020-08-08,18:05:14


In [76]:
yt['date_trending'] = pd.to_datetime(yt['trending_date']).dt.date
yt['date_trending'] = pd.to_datetime(yt['date_trending'])
yt[100:102]

,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,description,date_published,time_published,date_trending
100,WE DID IT,2020-08-09T19:54:58Z,Deji,24,2020-08-12T00:00:00Z,[None],1547751,232695,4065,12748,FOLLOW MY REDDIT: https://www.reddit.com/r/Dej...,2020-08-09,19:54:58,2020-08-12
101,"Game Theory: Minecraft, Stop Using Diamonds!",2020-08-08T18:05:14Z,The Game Theorists,20,2020-08-12T00:00:00Z,minecraft|minecraft theory|minecraft diamonds|...,2657891,154064,6509,21993,Thanks Honey for sponsoring! Get the best dea...,2020-08-08,18:05:14,2020-08-12


In [77]:
yt['time_trending'] = pd.to_datetime(yt['trending_date']).dt.time
yt[100:102]

,title,publishedAt,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,description,date_published,time_published,date_trending,time_trending
100,WE DID IT,2020-08-09T19:54:58Z,Deji,24,2020-08-12T00:00:00Z,[None],1547751,232695,4065,12748,FOLLOW MY REDDIT: https://www.reddit.com/r/Dej...,2020-08-09,19:54:58,2020-08-12,00:00:00
101,"Game Theory: Minecraft, Stop Using Diamonds!",2020-08-08T18:05:14Z,The Game Theorists,20,2020-08-12T00:00:00Z,minecraft|minecraft theory|minecraft diamonds|...,2657891,154064,6509,21993,Thanks Honey for sponsoring! Get the best dea...,2020-08-08,18:05:14,2020-08-12,00:00:00


Now, we can drop unnecessary columns that we used to convert!

In [78]:
yt.drop(columns=['publishedAt','trending_date'], inplace = True)

In [79]:
yt[100:102]

,title,channelTitle,categoryId,tags,view_count,likes,dislikes,comment_count,description,date_published,time_published,date_trending,time_trending
100,WE DID IT,Deji,24,[None],1547751,232695,4065,12748,FOLLOW MY REDDIT: https://www.reddit.com/r/Dej...,2020-08-09,19:54:58,2020-08-12,00:00:00
101,"Game Theory: Minecraft, Stop Using Diamonds!",The Game Theorists,20,minecraft|minecraft theory|minecraft diamonds|...,2657891,154064,6509,21993,Thanks Honey for sponsoring! Get the best dea...,2020-08-08,18:05:14,2020-08-12,00:00:00


----------------------------
5. Now, we want to use our dataset to extract months of those 2 columns we mentioned earlier, and create new columns based on them, which are:
- month_trending
- month_published

In [80]:
yt['month_trending'] = yt['date_trending'].dt.month
yt['month_published'] = yt['date_published'].dt.month

In [87]:
yt[800:802]

,title,channelTitle,categoryId,tags,view_count,likes,dislikes,comment_count,description,date_published,time_published,date_trending,time_trending,month_trending,month_published
800,"Last To Leave $800,000 Island Keeps It",MrBeast,24,[None],17365719,1599289,23874,166844,I can't believe I gave away a private island. ...,2020-08-15,20:00:02,2020-08-16,00:00:00,8,8
801,Drake - Laugh Now Cry Later (Official Music Vi...,DrakeVEVO,10,Drake|Laugh|Now|Cry|Later|OVO|Hip|Hop,17156758,1006096,24503,71191,Laugh Now Cry Later ft. Lil Durk available eve...,2020-08-14,04:00:08,2020-08-16,00:00:00,8,8


I checked many entries of this dataset, and I saw many entries with month 08 :)) that means something has anything to do with August :))

#### Replacing [8,9,10,11] to ['Aug','Sept','Oct','Nov']:

In [ ]:
yt['month_trending'].replace([8,9,10,11], ['Aug','Sept','Oct','Nov'], inplace=True)
yt['month_published'].replace([8,9,10,11], ['Aug','Sept','Oct','Nov'], inplace=True)